In [31]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

# Step 1: Initialize a SparkSession
spark = SparkSession.builder.master("local[*]") \
    .appName("LocalRowCountFoo") \
    .getOrCreate()


# # Step 2: Create an RDD with the "Hello World" message
# data = ["Hello", "World"]
# rdd = spark.sparkContext.parallelize(data)

# # Step 3: Perform an action to collect and print the data
# result = rdd.collect()
# print(" ".join(result))

# # Step 4: Stop the Spark session
# spark.stop()


In [32]:
df = spark.read.csv('dummy_data.csv',header=True,inferSchema=True)
df.createOrReplaceTempView('customer')
df.show()

+---+---------+---------+---+--------------------+--------------------+
| ID|FirstName| LastName|Age|               Email|             Country|
+---+---------+---------+---+--------------------+--------------------+
|  1|Elizabeth|  Bennett| 32|elizabeth.bennett...|               Kenya|
|  2| Benjamin|   Garcia| 28|benjamin.garcia@s...|         Switzerland|
|  3|Elizabeth|   Jordan| 56|elizabeth.jordan@...|              Poland|
|  4|Charlotte|    Scott| 59|charlotte.scott@d...|              Greece|
|  5|  William|   Carter| 47|william.carter@de...|              Russia|
|  6|     Noah|    Adams| 40|noah.adams@sample...|           Australia|
|  7|     Lucy| Phillips| 42|lucy.phillips@dem...|             Belgium|
|  8|   Thomas|   Miller| 34|thomas.miller@sam...|           Indonesia|
|  9|   Thomas|   Murphy| 36|thomas.murphy@exa...|               China|
| 10|    Layla|  Collins| 19|layla.collins@sam...|           Argentina|
| 11|   Thomas|     Hall| 28|thomas.hall@examp...|              

In [33]:
sql_counts ='''
select 
    country, 
    count(*) as customer_count
from customer 
group by country
'''
sql_df = spark.sql(sql_counts)
sql_df.show()
sql_df.createOrReplaceTempView('customer_counts')

+-------------+--------------+
|      country|customer_count|
+-------------+--------------+
|       Russia|       2000579|
|       Sweden|       2001276|
|  Philippines|       1999695|
|    Singapore|       1998271|
|     Malaysia|       2000236|
|       Turkey|       1997944|
|      Germany|       2001110|
|       France|       2000593|
|       Greece|       1999727|
|    Argentina|       3998312|
|      Belgium|       2001571|
|      Finland|       1997907|
|         Peru|       2001010|
|        China|       2000004|
|        India|       2001237|
|United States|       1999854|
|        Chile|       2000366|
|      Nigeria|       1997912|
|        Italy|       1998986|
|       Norway|       1996855|
+-------------+--------------+
only showing top 20 rows



In [34]:
sql_stats ='''

    SELECT 
        AVG(customer_count) AS avg_count, 
        STDDEV(customer_count) AS stddev_count,
        sum(customer_count) as total_customers
    FROM customer_counts

'''
avg_df = spark.sql(sql_stats)
avg_df.show()
avg_df.createOrReplaceTempView('customer_stats')

+------------------+------------------+---------------+
|         avg_count|      stddev_count|total_customers|
+------------------+------------------+---------------+
|2040816.3265306123|285470.89293199603|      100000000|
+------------------+------------------+---------------+



In [35]:

sql_z ='''
SELECT 
    c.country,
    c.customer_count,
    (c.customer_count - s.avg_count) / s.stddev_count AS z_score,
    case when abs((c.customer_count - s.avg_count) / s.stddev_count) > 3 then 1 else 0 end   AS is_suspect
FROM customer_counts c
CROSS JOIN customer_stats s;
'''
z_df = spark.sql(sql_z)
z_df.show()

+-------------+--------------+--------------------+----------+
|      country|customer_count|             z_score|is_suspect|
+-------------+--------------+--------------------+----------+
|       Russia|       2000579| -0.1409507152107361|         0|
|       Sweden|       2001276|-0.13850913529048117|         0|
|  Philippines|       1999695|-0.14404735315837647|         0|
|    Singapore|       1998271|-0.14903560252199624|         0|
|     Malaysia|       2000236| -0.1421522387582934|         0|
|       Turkey|       1997944|-0.15018107832389488|         0|
|      Germany|       2001110|-0.13909063065168967|         0|
|       France|       2000593|-0.14090167343328477|         0|
|       Greece|       1999727|-0.14393525766705917|         0|
|    Argentina|       3998312|   6.857076227157408|         1|
|      Belgium|       2001571|-0.13747575497989983|         0|
|      Finland|       1997907| -0.1503106887357305|         0|
|         Peru|       2001010|-0.13944092906205624|    

In [36]:
spark.stop()